# Lab 2 - TSP

The Traveling Salesman Problem asks about finding the shortest possible route that visits each cities (from a known list) exactly once and returns to the origin city. 

So we are looking for a minimum cost hamiltonian cycle, inside a fully connected graph.

In [1]:
## Import

import logging
import random
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [34]:
## Loading cities from the directory

CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])

## Initializing the distance matrix -> distance between couple of cities
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
    
CITIES.head()

,name,lat,lon
0,Abilene,32.454514,-99.738147
1,Akron,41.080456,-81.521429
2,Albuquerque,35.105552,-106.647388
3,Alexandria,38.818343,-77.082026
4,Allen,33.107224,-96.674676


In [35]:
##Defining the cost

def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## First Greedy Solution 

## Second Solution - EA with inver over

For the second solution for the TSP i choose to implement an evolutionary algorithm using the inver over. 
The inver over is based on the idea that one genes are selected in the first parent, on edge is taken from the second parent, trying to preserve edges of the first.



The inver-over operator is particularly useful for certain types of combinatorial optimization problems, like the TSP, because it effectively preserves and improves the "local structures" of the solution, such as the sequence of cities in a route.

### Defining some functions

I defined some useful functions:
- fitness(): fitness function 
- generate_random_tour(): generating a randomic tour of the cities from the csv file. 
- inver_over(): reverses sections of the current route, trying to find a city configuration with a shorter overall distance.
- roulette_selection(): 

In [36]:
#defining the fitness function
def fitness(tour):
    return 1/tsp_cost(tour)


#genereting a randomic tour of the cities
def generate_random_tour():
    tour = list(range(len(CITIES)))
    random.shuffle(tour)
    tour.append(tour[0])
    return tour


#defining the inver_over mutation
def inver_over(tour):
    new_tour = tour[:-1] 
    start = random.randint(0, len(new_tour) - 1 )
    end = random.randint(0, len(new_tour) -1 )
    
    if start > end:
        start, end = end, start
    new_tour[start:end+1] =  reversed(new_tour[start:end+1])
    new_tour.append(new_tour[0])
    
    return new_tour

#defining the roulette selection to find the parents from a population of candidates
def roulette_selection(population, fitness_values):
    max_fit = sum(fitness_values)
    pick = random.uniform(0, max_fit)
    current = 0
    for i, fit in enumerate(fitness_values):
        current += fit
        if current > pick:
            return population[i]
    return population[-1]



In [37]:
##Defining the genetic algorithm

def genetic_algorithm(cities, dist_matrix, pop_size=100, generations=1000):
    population = [generate_random_tour() for _ in range(pop_size)]
    best_solution = min(population, key=lambda x: tsp_cost(x))
    best_cost = tsp_cost(best_solution)
    best_generation = 0

    for generation in range(generations):
        fitness_values = [fitness(tour) for tour in population]
        new_population = []
        
        #Generating new population 
        for _ in range(pop_size):
            parent = roulette_selection(population, fitness_values)
            child = inver_over(parent)
            new_population.append(child)
        
        #Updating the population
        population = new_population
        candidate_best = min(population, key=lambda x: tsp_cost(x))
        candidate_cost = tsp_cost(candidate_best)
        
        if candidate_cost < best_cost:
            best_solution = candidate_best
            best_cost = candidate_cost
            best_generation = generation

        # Log delle performance ogni 100 generazioni
        if generation % 100 == 0:
            logging.debug(f"Generation {generation}: Best cost = {best_cost:.2f}")

    # Restituisci anche i nomi delle città nel miglior tour
    best_city_names = [cities.iloc[i]['name'] for i in best_solution]
    return best_city_names, best_cost, best_generation

TSP - Vanuatu

In [40]:
## Loading cities from the directory

CITIES = pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])

## Initializing the distance matrix -> distance between couple of cities
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

In [41]:
solution, final_cost, steps = genetic_algorithm(CITIES, DIST_MATRIX, pop_size=100, generations=1000)

logging.info(f"Best tour: {solution}")  #Shows the best path/tour of the cities
logging.info(f"Total distance: {final_cost:.2f} km")  # Shows the total distance 
logging.info(f"Steps taken: {steps}")  #Steps taken to find the best solution

DEBUG:root:Generation 0: Best cost = 1345.54
DEBUG:root:Generation 100: Best cost = 1345.54
DEBUG:root:Generation 200: Best cost = 1345.54
DEBUG:root:Generation 300: Best cost = 1345.54
DEBUG:root:Generation 400: Best cost = 1345.54
DEBUG:root:Generation 500: Best cost = 1345.54
DEBUG:root:Generation 600: Best cost = 1345.54
DEBUG:root:Generation 700: Best cost = 1345.54
DEBUG:root:Generation 800: Best cost = 1345.54
DEBUG:root:Generation 900: Best cost = 1345.54
INFO:root:Best tour: ['Vila', 'Isangel', 'Longana', 'Sola', 'Port Olry', 'Luganville', 'Norsup', 'Lakatoro', 'Vila']
INFO:root:Total distance: 1345.54 km
INFO:root:Steps taken: 18


TSP - Italy

In [43]:
## Loading cities from the directory

CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])

## Initializing the distance matrix -> distance between couple of cities
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
    


In [44]:
solution, final_cost, steps = genetic_algorithm(CITIES, DIST_MATRIX, pop_size=100, generations=1000)

logging.info(f"Best tour: {solution}")  #Shows the best path/tour of the cities
logging.info(f"Total distance: {final_cost:.2f} km")  # Shows the total distance 
logging.info(f"Steps taken: {steps}")  #Steps taken to find the best solution

DEBUG:root:Generation 0: Best cost = 15491.01
DEBUG:root:Generation 100: Best cost = 12348.23
DEBUG:root:Generation 200: Best cost = 12348.23
DEBUG:root:Generation 300: Best cost = 12348.23
DEBUG:root:Generation 400: Best cost = 12348.23
DEBUG:root:Generation 500: Best cost = 12348.23
DEBUG:root:Generation 600: Best cost = 12348.23
DEBUG:root:Generation 700: Best cost = 12348.23
DEBUG:root:Generation 800: Best cost = 12348.23
DEBUG:root:Generation 900: Best cost = 12348.23
INFO:root:Best tour: ['Perugia', 'Ancona', 'Cagliari', 'Palermo', 'Sassari', 'Prato', 'Rome', 'Terni', 'Bologna', 'Rimini', 'Pescara', 'Foggia', 'Naples', 'Florence', 'Padua', 'Trento', 'Latina', 'Leghorn', 'Forlì', 'Taranto', 'Reggio di Calabria', 'Bari', 'Salerno', 'Messina', 'Catania', 'Giugliano in Campania', 'Andria', 'Vicenza', 'Brescia', 'Bolzano', 'Modena', 'Genoa', "Reggio nell'Emilia", 'Venice', 'Verona', 'Ravenna', 'Syracuse', 'Piacenza', 'Novara', 'Turin', 'Ferrara', 'Trieste', 'Milan', 'Monza', 'Bergamo'

TSP - Russia

In [45]:
## Loading cities from the directory

CITIES = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])

## Initializing the distance matrix -> distance between couple of cities
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
    

In [46]:
solution, final_cost, steps = genetic_algorithm(CITIES, DIST_MATRIX, pop_size=100, generations=1000)

logging.info(f"Best tour: {solution}")  #Shows the best path/tour of the cities
logging.info(f"Total distance: {final_cost:.2f} km")  # Shows the total distance 
logging.info(f"Steps taken: {steps}")  #Steps taken to find the best solution

DEBUG:root:Generation 0: Best cost = 302689.22
DEBUG:root:Generation 100: Best cost = 286099.78
DEBUG:root:Generation 200: Best cost = 284140.35
DEBUG:root:Generation 300: Best cost = 281649.96
DEBUG:root:Generation 400: Best cost = 269260.59
DEBUG:root:Generation 500: Best cost = 269260.59
DEBUG:root:Generation 600: Best cost = 265501.12
DEBUG:root:Generation 700: Best cost = 254754.14
DEBUG:root:Generation 800: Best cost = 252748.78
DEBUG:root:Generation 900: Best cost = 252748.78
INFO:root:Best tour: ['Komsomolsk‐na‐Amure', 'Tobolsk', 'Seversk', 'Magadan', 'Blagoveshchensk', 'Ussuriysk', 'Vladivostok', 'Yuzhno‐Sakhalinsk', 'Khabarovsk', 'Velikiy Novgorod', 'Cheboksary', 'Novoshakhtinsk', 'Noginsk', 'Novokuybyshevsk', 'Balakovo', 'Korolyov', 'Nalchik', 'Almetyevsk', 'Kolomna', 'Ulyanovsk', 'Kostroma', 'Kamensk‐Uralskiy', 'Tyumen', 'Obninsk', 'Groznyy', 'Nevinnomyssk', 'Severodvinsk', 'Armavir', 'Nizhnevartovsk', 'Kamyshin', 'Zelenograd', 'Petrozavodsk', 'Lyubertsy', 'Vladimir', 'Rybi

TSP - US

In [47]:
## Loading cities from the directory

CITIES = pd.read_csv('cities/us.csv', header=None, names=['name', 'lat', 'lon'])

## Initializing the distance matrix -> distance between couple of cities
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

In [48]:
solution, final_cost, steps = genetic_algorithm(CITIES, DIST_MATRIX, pop_size=100, generations=1000)

logging.info(f"Best tour: {solution}")  #Shows the best path/tour of the cities
logging.info(f"Total distance: {final_cost:.2f} km")  # Shows the total distance 
logging.info(f"Steps taken: {steps}")  #Steps taken to find the best solution

DEBUG:root:Generation 0: Best cost = 585251.25
DEBUG:root:Generation 100: Best cost = 562834.85
DEBUG:root:Generation 200: Best cost = 562834.85
DEBUG:root:Generation 300: Best cost = 562834.85
DEBUG:root:Generation 400: Best cost = 562834.85
DEBUG:root:Generation 500: Best cost = 562834.85
DEBUG:root:Generation 600: Best cost = 562834.85
DEBUG:root:Generation 700: Best cost = 562834.85
DEBUG:root:Generation 800: Best cost = 562834.85
DEBUG:root:Generation 900: Best cost = 562834.85
INFO:root:Best tour: ['Broken Arrow', 'Lewisville', 'Jurupa Valley', 'Sparks', 'Worcester', 'Boise', 'Dallas', 'Los Angeles', 'Tallahassee', 'Indianapolis', 'Bakersfield', 'Hollywood', 'Sioux Falls', 'Sandy Springs', "Lee's Summit", 'Highlands Ranch', 'Corona', 'Plano', 'Henderson', 'Athens', 'Garden Grove', 'Irvine', 'Oxnard', 'Ontario', 'Thornton', 'College Station', 'El Paso', 'Odessa', 'New York City', 'Spring Hill', 'Sunrise Manor', 'Norfolk', 'West Palm Beach', 'Lexington', 'Santa Clara', 'Denver', 'W